In [1]:
import pandas as pd
from utils_io import load_data
from cairosvg import svg2pdf

In [ ]:
## 1. Supp_Fig 1d  ---  --- 

In [2]:
def shape_to_color_v5(shape):
    if shape < -998:
        return "rgb(100%, 100%, 100%)"
    else:
        if shape >0.85:
            return 'rgb(90%,0%,20%)'
        elif shape > 0.4:
            return 'rgb(255,127,0)'
        else:
            return 'rgb(128,128,128)'     

In [3]:
### A. get the normalized reactivities ---

bench = pd.read_csv('/home/han/proj_het_AC/rerun_analysis/0_Manuscript_codes_submission_20250723/Supp_Fig1/Data/Tetrahymena.pairing.txt', sep=',')
bench['pos_0_based'] = bench['pos']-1

flt=bench
seq = ''.join(flt['nt'].astype(str))
ss = ''.join(flt['dbrt'].astype(str))
print(len(seq))
print(len(ss))
seq = seq
ss=ss

def load_profile(p_gene, p_batch, 
                 p_method="Nanopolish", p_nu_gamma="nu_9e-4_gamma_1e-2", 
                 p_threshold_len_prop=0.7, f_sizes=None):

    dir = "/home/han/proj_het_AC/rerun_analysis/0_Manuscript_codes_submission_20250723/Supp_Fig1/Data/"
    f_mtxs = [dir + "/TETRA-WT_nanopolish+porecupine_binary.mtx"]
    f_ids = [dir + "/TETRA-WT_nanopolish+porecupine_binary.iids.gz"]
    X, dfs = load_data(f_mtxs, f_ids, [p_gene], f_sizes=f_sizes, p_depth=-1, p_length=-1, 
                       p_start=None, p_end=None, p_threshold=1, 
                       p_threshold_len_prop=p_threshold_len_prop, p_verbose=False)
    df, = dfs

    modrate = (df.sum().sum()*100)/(df.shape[0]*df.shape[1])
    prop = df.sum(axis=0)/df.shape[0]
    p_batch = p_batch.replace("_30As", "")

    return (p_gene, "WT", p_method.split("+")[0], p_batch, df.shape[0], modrate, prop)


f_sizes = "/home/han/proj_het_AC/rerun_analysis/0_Manuscript_codes_submission_20250723/Supp_Fig1/Data/reference_of_ribosxitch_v2.sizes"

list_of_genes = ["TETRA-WT"]

list_of_batches = [
                   ["NAIN3", "230314_12RNAs_30As_50mMNAIN3_WTpool", "nu_9e-4_gamma_1e-2"]
                   ]



dict_profiles = {}
for p_gene in list_of_genes:
    for p_mod, p_batch, p_ng_np in list_of_batches:
        dict_profiles[p_batch.replace("_30As", "")] = load_profile(p_gene, p_batch, p_method="Nanopolish", 
                                                                   p_nu_gamma=p_ng_np, f_sizes=f_sizes)

nain3_50 = dict_profiles['230314_12RNAs_50mMNAIN3_WTpool'][6]
mod = nain3_50 

mod = pd.DataFrame(mod)
mod.columns = ['mod_rate']
mod['position'] = mod.index 
mod['position'] = mod['position'].astype(int)
mod = mod.reset_index()
mod['position'] = mod.index 
mod = mod[['position','mod_rate']]
mod = mod[['mod_rate']]
mod['mutrate'] = mod['mod_rate']

### to get the normalized reactivity

reactivity_flt = mod[mod['mutrate'].notnull()] 

per_25 = reactivity_flt.mutrate.quantile(0.25)
per_75 = reactivity_flt.mutrate.quantile(0.75)
per_90 = reactivity_flt.mutrate.quantile(0.90)
interquartile = per_75 - per_25
max_cutoff =  max(interquartile*1.5, per_90)
 
flt_above_max = reactivity_flt[(reactivity_flt['mutrate'] >= max_cutoff)] 
flt_below_max = reactivity_flt[(reactivity_flt['mutrate'] < max_cutoff)] 
   
flt_per_90 = flt_below_max.mutrate.quantile(0.90)
flt_per_100 = flt_below_max.mutrate.quantile(1.00)
flt_remain_10 = flt_below_max[(flt_below_max['mutrate'] < flt_per_100) & (flt_below_max['mutrate'] > flt_per_90)] 
mean_remain_10 = flt_remain_10.mutrate.mean()
print(mean_remain_10)   

mean_remain_10=0.020
norm_value = (mod.mutrate/mean_remain_10).tolist()
print(len(norm_value))
mod['Norm_reactivity'] = norm_value
mod_flt = list(mod['Norm_reactivity'])
modrate = list(mod_flt[28:])
print(len(modrate)) 

393
393
(15562, 3)
15562
0.006306099345382598
421
393


In [4]:
### B. mapped the structures --- ---

circ_colors = [shape_to_color_v5(s) for s in modrate]
print(len(circ_colors))
f = open("/home/han/proj_het_AC/rerun_analysis/0_Manuscript_codes_submission_20250723/Supp_Fig1/Data/tetrahymena_from_ashley.in_vivo.WT.svg").readlines() 
mapped_svg = open("/home/han/proj_het_AC/rerun_analysis/0_Manuscript_codes_submission_20250723/Supp_Fig1/Data/Supp_Fig1d_tetrahymena.colored.Nanopolish.svg","w+")
lines_number = 0

count=0
for line in f:

    if "style=" in line:

        count+=1
        print(count)
        loca = count-1
        print(count)

        circ_color_str = circ_colors[loca]
        circ_insert  = f'"fill:{circ_color_str};stroke:none; stroke-width:1; fill-opacity:0.8"'
        
        # create the circ insert lines 
        circ_pos_lines = "   style="+ circ_insert + "\n"
        
        mapped_svg.write(circ_pos_lines)
       
    else: mapped_svg.write(line)

lines_number+=1

mapped_svg.close() 

### convert to PDF ------------

dir_out ="/home/han/proj_het_AC/rerun_analysis/0_Manuscript_codes_submission_20250723/Supp_Fig1/Data/"
from cairosvg import svg2pdf
svg2pdf(url= dir_out+'Supp_Fig1d_tetrahymena.colored.Nanopolish.svg', write_to=dir_out+'Supp_Fig1d_tetrahymena.colored.Nanopolish.pdf', dpi=300)


393
1
1
2
2
3
3
4
4
5
5
6
6
7
7
8
8
9
9
10
10
11
11
12
12
13
13
14
14
15
15
16
16
17
17
18
18
19
19
20
20
21
21
22
22
23
23
24
24
25
25
26
26
27
27
28
28
29
29
30
30
31
31
32
32
33
33
34
34
35
35
36
36
37
37
38
38
39
39
40
40
41
41
42
42
43
43
44
44
45
45
46
46
47
47
48
48
49
49
50
50
51
51
52
52
53
53
54
54
55
55
56
56
57
57
58
58
59
59
60
60
61
61
62
62
63
63
64
64
65
65
66
66
67
67
68
68
69
69
70
70
71
71
72
72
73
73
74
74
75
75
76
76
77
77
78
78
79
79
80
80
81
81
82
82
83
83
84
84
85
85
86
86
87
87
88
88
89
89
90
90
91
91
92
92
93
93
94
94
95
95
96
96
97
97
98
98
99
99
100
100
101
101
102
102
103
103
104
104
105
105
106
106
107
107
108
108
109
109
110
110
111
111
112
112
113
113
114
114
115
115
116
116
117
117
118
118
119
119
120
120
121
121
122
122
123
123
124
124
125
125
126
126
127
127
128
128
129
129
130
130
131
131
132
132
133
133
134
134
135
135
136
136
137
137
138
138
139
139
140
140
141
141
142
142
143
143
144
144
145
145
146
146
147
147
148
148
149
149
150
150
151
151
152
